In [1]:
import pandas as pd
import pickle as pkl
from collections import Counter
import numpy as np

In [2]:
import os
os.chdir('../..')

In [3]:
parsed = pd.read_pickle('cache/final_vk_1500.pkl')

In [4]:
tsv_example = pd.read_csv('cache/example_clickbait.tsv', sep='\t')
tsv_example

,INPUT:headline,GOLDEN:result,HINT:text
0,Use this instead of butter to maked your grill...,yes,NaN
1,Notting Hill carnival partners with Glastonbur...,no,NaN
2,The seven best films to watch on TV this week,no,NaN
3,Someone has noticed something very wrong with ...,yes,NaN
4,"John Smith's real name isn't actually John, an...",yes,NaN
5,People are doing something seriously weird wit...,yes,NaN
6,A school girl gave her lunch to a homeless man...,NaN,NaN
7,Vanessa started drinking two glasses of bitter...,NaN,NaN
8,Hubble Captures Magnificent Grand Spiral Galax...,NaN,NaN
9,Only the people with an IQ above 160 can solve...,NaN,NaN


In [5]:
parsed.iloc[0]['json']['products']

[{'name': 'Сережки', 'price': 60, 'currency': 'рубли', 'count': 1}]

In [6]:
tasks = pd.DataFrame([{'text':row[1]['text'], 'product':product} for row in parsed.iterrows() for product in row[1]['json']['products']])
tasks

,text,product
0,Распродажа \nСережки \nЦена: 60р\nУпк от 10шт ...,"{'name': 'Сережки', 'price': 60, 'currency': '..."
1,Распродажа \nЦена: 150р \nЦена уп 5×140р\nРазм...,"{'name': 'Распродажа', 'price': 150, 'currency..."
2,Распродажа \nЦена: 150р \nЦена уп 5×140р\nРазм...,"{'name': 'Распродажа', 'price': 140, 'currency..."
3,Распродажа\nРост:86-92-98-104-110-116\nЦена: 5...,"{'name': '1-5-20 корпус А', 'price': 50, 'curr..."
4,Распродажа \nКрюч \nЦена:35р\nАрт:9-46,"{'name': 'Крюч', 'price': 35, 'currency': 'руб..."
...,...,...
3906,"Распродажа\nНабор канцелярский в стакане ""МУЛЬ...","{'name': 'ластик', 'price': None, 'currency': ..."
3907,"Распродажа\nНабор канцелярский в стакане ""МУЛЬ...","{'name': 'точилка', 'price': None, 'currency':..."
3908,Распродажа\nНаволочки\nЦена: 70р\nЦена упк: 20...,"{'name': 'Наволочка', 'price': 70, 'currency':..."
3909,Распродажа\nНаволочки\nЦена: 70р\nЦена упк: 20...,"{'name': 'Наволочка', 'price': 60, 'currency':..."


In [7]:
attr = 'count'
attr_type = [type(obj[attr]) if attr in obj else None for obj in tasks['product'].values]
Counter(attr_type)

Counter({int: 3558, None: 197, NoneType: 109, str: 46, float: 1})

In [8]:
tasks = tasks[[len(obj['name']) != 0 for obj in tasks['product'].values]]

In [9]:
tasks.loc[:,'product_name'] = [obj['name'] for obj in tasks['product'].values]

/tmp/ipykernel_169309/646972466.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tasks.loc[:,'product_name'] = [obj['name'] for obj in tasks['product'].values]


In [10]:
tasks[[(type(obj['price']) if 'price' in obj else None) == str for obj in tasks['product']]]['product'].values[4]
# Counter([type(obj['price']) if 'price' in obj else None for obj in tasks['product']])

{'name': 'Gonyosoma boulengeri', 'price': '3.4 СВ20'}

In [11]:
mask = [('price' in obj) and obj['price'] is not None for obj in tasks['product'].values]
tasks = tasks[mask]

In [12]:
def price_handler(product):
  price = product['price']
  if type(price) not in [int, float]:
    try:
      price = int(price)
    except:
      price = None
  return str(price)
tasks.loc[:, ['product_price']] = tasks['product'].apply(price_handler)

In [13]:
tasks = tasks[~tasks.product_price.isna()]

In [14]:
tasks

,text,product,product_name,product_price
0,Распродажа \nСережки \nЦена: 60р\nУпк от 10шт ...,"{'name': 'Сережки', 'price': 60, 'currency': '...",Сережки,60
1,Распродажа \nЦена: 150р \nЦена уп 5×140р\nРазм...,"{'name': 'Распродажа', 'price': 150, 'currency...",Распродажа,150
2,Распродажа \nЦена: 150р \nЦена уп 5×140р\nРазм...,"{'name': 'Распродажа', 'price': 140, 'currency...",Распродажа,140
3,Распродажа\nРост:86-92-98-104-110-116\nЦена: 5...,"{'name': '1-5-20 корпус А', 'price': 50, 'curr...",1-5-20 корпус А,50
4,Распродажа \nКрюч \nЦена:35р\nАрт:9-46,"{'name': 'Крюч', 'price': 35, 'currency': 'руб...",Крюч,35
...,...,...,...,...
3901,Распродажа \nКофточка Водолазки\nТкань: шерсть...,"{'name': 'Кофточка Водолазки', 'price': 300, '...",Кофточка Водолазки,300
3902,"Распродажа\nНабор канцелярский в стакане ""МУЛЬ...",{'name': 'набор канцелярский в стакане МУЛЬТЯШ...,набор канцелярский в стакане МУЛЬТЯШКИ,100
3908,Распродажа\nНаволочки\nЦена: 70р\nЦена упк: 20...,"{'name': 'Наволочка', 'price': 70, 'currency':...",Наволочка,70
3909,Распродажа\nНаволочки\nЦена: 70р\nЦена упк: 20...,"{'name': 'Наволочка', 'price': 60, 'currency':...",Наволочка,60


In [28]:
def currency_handler(product):
  currency = None
  if currency in product:
    currency = product['currency']
  if currency in ['рубли', '₽', None]:
    currency = 'RUB'
  return currency

tasks['product_currency'] = tasks['product'].apply(currency_handler)

/tmp/ipykernel_169309/3940525848.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tasks['product_currency'] = tasks['product'].apply(currency_handler)


In [29]:
def count_handler(product):
  count = None
  if 'count' in product:
    count = product['count']
  if '' == count:
    count = None
  return count

tasks['product_count'] = tasks['product'].apply(count_handler).values
tasks = tasks[~tasks['product_count'].isna()]

/tmp/ipykernel_169309/734486376.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tasks['product_count'] = tasks['product'].apply(count_handler).values


In [80]:
import json

In [81]:
tasks['product'] = tasks['product'].apply(lambda x: json.dumps(x))

In [91]:
len(tasks.drop_duplicates())

3502

In [17]:
tsv_tasks = tasks.drop(columns=['product'])

In [19]:
tsv_tasks = tsv_tasks.add_prefix('INPUT:')

In [20]:
tsv_example

,INPUT:headline,GOLDEN:result,HINT:text
0,Use this instead of butter to maked your grill...,yes,NaN
1,Notting Hill carnival partners with Glastonbur...,no,NaN
2,The seven best films to watch on TV this week,no,NaN
3,Someone has noticed something very wrong with ...,yes,NaN
4,"John Smith's real name isn't actually John, an...",yes,NaN
5,People are doing something seriously weird wit...,yes,NaN
6,A school girl gave her lunch to a homeless man...,NaN,NaN
7,Vanessa started drinking two glasses of bitter...,NaN,NaN
8,Hubble Captures Magnificent Grand Spiral Galax...,NaN,NaN
9,Only the people with an IQ above 160 can solve...,NaN,NaN


In [20]:
tsv_tasks.to_csv('cache/vk_toloka.tsv', sep='\t', index=False)

In [57]:
import json

In [71]:
tasks['product']

0       {'name': 'Сережки', 'price': 60, 'currency': '...
1       {'name': 'Распродажа', 'price': 150, 'currency...
2       {'name': 'Распродажа', 'price': 140, 'currency...
4       {'name': 'Крюч', 'price': 35, 'currency': 'руб...
5       {'name': 'Распродажа', 'price': 300, 'currency...
                              ...                        
3901    {'name': 'Кофточка Водолазки', 'price': 300, '...
3902    {'name': 'набор канцелярский в стакане МУЛЬТЯШ...
3908    {'name': 'Наволочка', 'price': 70, 'currency':...
3909    {'name': 'Наволочка', 'price': 60, 'currency':...
3910    {'name': 'Ткань хлопок', 'price': 50, 'currenc...
Name: product, Length: 3544, dtype: object

In [86]:
ls = [(row[0], json.dumps(prod), json.dumps({k:v for k,v in prod.items() if k in ['name', 'price', 'currency', 'count']})) for row in parsed.values for prod in row[2]['products']]

In [87]:
count = 0
for i, l1 in enumerate(ls):
  for j, l2 in enumerate(ls):
    if l1 == l2:
      continue
    if (l1[0] == l2[0]) and (l1[1] == l2[1]):
      print(i, j)

In [97]:
bad_text = tsv_tasks[tsv_tasks.duplicated()].values[0][0]

In [103]:
parsed[parsed.text == bad_text].json.values[0]

{'place': '',
 'products': [{'name': 'Распродажа',
   'price': 100,
   'currency': 'рубли',
   'size': '42',
   'count': 1},
  {'name': 'Распродажа',
   'price': 100,
   'currency': 'рубли',
   'size': '44',
   'count': 1},
  {'name': 'Распродажа',
   'price': 100,
   'currency': 'рубли',
   'size': '46',
   'count': 1},
  {'name': 'Распродажа',
   'price': 100,
   'currency': 'рубли',
   'size': '48',
   'count': 1},
  {'name': 'Распродажа',
   'price': 100,
   'currency': 'рубли',
   'size': '50',
   'count': 1},
  {'name': 'Распродажа',
   'price': 100,
   'currency': 'рубли',
   'size': '52',
   'count': 1}]}